# Web Search quickstart

Demonstrates Azure OpenAI Responses API with the `web_search_preview` tool using environment variables loaded via `.env`.


## Prerequisites
- Install `openai` (>=1.56.0) and `python-dotenv`.
- Populate `.env` with your Azure OpenAI settings. Supported keys (first match wins):
  - Base URL: `AZURE_OPENAI_BASE_URL` | `AZURE_OPENAI_API_BASE` | `AZURE_EXISTING_AIPROJECT_ENDPOINT`
  - API key: `AZURE_OPENAI_API_KEY` | `OPENAI_API_KEY`
  - Model deployment (optional): `AZURE_OPENAI_MODEL` | `AZURE_OPENAI_DEPLOYMENT` (default `gpt-4.1`)
  - Country code (optional): `AZURE_OPENAI_COUNTRY` (for location-biased search)
  - Deep research deployment (optional): `AZURE_OPENAI_DEEP_MODEL`
- Using web search can incur costs and sends data to Bing for grounding.


In [ ]:
# If needed, install dependencies (uncomment to run)
# %pip install -U openai python-dotenv


In [20]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

base_url = (
    os.getenv("AZURE_OPENAI_BASE_URL")
    or os.getenv("AZURE_OPENAI_API_BASE")
    or os.getenv("AZURE_EXISTING_AIPROJECT_ENDPOINT")
)
api_key = os.getenv("AZURE_OPENAI_API_KEY") or os.getenv("OPENAI_API_KEY")
model = os.getenv("AZURE_OPENAI_MODEL") or os.getenv("AZURE_OPENAI_DEPLOYMENT") or "gpt-4.1-mini"
print(model)
country = os.getenv("AZURE_OPENAI_COUNTRY")
deep_model = os.getenv("AZURE_OPENAI_DEEP_MODEL")
print(deep_model)

if not base_url or not api_key:
    raise ValueError("Set base URL and API key in .env (see prerequisites).")

base_url = base_url.rstrip("/")
if not base_url.endswith("/openai/v1"):
    base_url = f"{base_url}/openai/v1"


gpt-4.1-mini
o3-pro


In [21]:
from openai import OpenAI

client = OpenAI(api_key=api_key, base_url=base_url)
print(f"Client ready against {base_url}")


Client ready against https://eu2-oai.openai.azure.com/openai/v1


In [22]:
#Formatting code
import json
from typing import Optional, List

def run_search(prompt: str, tools: List[dict], model_override: Optional[str] = None):
    response = client.responses.create(
        model=model_override or model,
        tools=tools,
        input=prompt,
    )

    print(response.output_text)

    # Display annotations (citations) if present
    messages = [item for item in response.output or [] if item.type == "message"]
    for message in messages:
        for content in message.content or []:
            if hasattr(content, 'annotations') and content.annotations:
                print("\n" + "=" * 80)
                print("Citations:")
                print("=" * 80)
                for i, annotation in enumerate(content.annotations, 1):
                    # Extract the cited text using start_index and end_index
                    if hasattr(annotation, 'start_index') and hasattr(annotation, 'end_index'):
                        cited_text = content.text[annotation.start_index:annotation.end_index]
                        print(f"\n[{i}] {cited_text}")
                    else:
                        print(f"\n[{i}] Citation {i}")
                    
                    # Display URL and title if available
                    if hasattr(annotation, 'url') and annotation.url:
                        print(f"    URL: {annotation.url}")
                    if hasattr(annotation, 'title') and annotation.title:
                        print(f"    Title: {annotation.title}")
                print("=" * 80)

    actions = [item for item in response.output or [] if item.type == "web_search_call"]
    if actions:
        print("\n" + "=" * 80)
        print("Tool calls:")
        print("=" * 80)
        for i, action in enumerate(actions, 1):
            try:
                print(f"\nCall {i}:")
                print(json.dumps(action.model_dump(), indent=2, ensure_ascii=False, sort_keys=False))
            except Exception:
                print(action)
        print("=" * 80)

    return response


### Quick lookup (single search)
Uses the fastest path: the model forwards the prompt directly to web search without multi-step planning.


In [24]:
run_search(
    "What happened in the last NFL game?",
    tools=[{"type": "web_search_preview"}],
)

The last NFL game was the Wild Card playoff between the Los Angeles Rams and the Carolina Panthers, where the Rams won 34-31. This game occurred on January 10, 2026. There were other recent playoff games as well, including the Bears beating the Packers 31-27 and the Bills winning against the Jaguars 27-24.

For the latest ongoing games in the Divisional Round playoffs:
- Buffalo Bills vs. Denver Broncos on January 17, 2026
- San Francisco 49ers vs. Seattle Seahawks on January 17, 2026
- Houston Texans vs. New England Patriots on January 18, 2026
- Los Angeles Rams vs. Chicago Bears on January 18, 2026

If you want details on a specific recent game or ongoing one, let me know! 

Source: [CBSSports NFL Scores](https://www.cbssports.com/nfl/scoreboard/all/)

Citations:

[1] [CBSSports NFL Scores](https://www.cbssports.com/nfl/scoreboard/all/)
    URL: https://www.cbssports.com/nfl/scoreboard/all/
    Title: NFL Game Scores - - CBS Sports

Tool calls:

Call 1:
{
  "id": "ws_041f129d557177e

Response(id='resp_041f129d557177e10069694e3faa708190ae74fd94f595c6f5', created_at=1768508991.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4.1-mini', object='response', output=[ResponseFunctionWebSearch(id='ws_041f129d557177e10069694e40e5b881908bceb0c06c067905', action=ActionSearch(query='latest NFL game results', type='search', sources=None), status='searching', type='web_search_call'), ResponseOutputMessage(id='msg_041f129d557177e10069694e41c6c88190a803db0bda2858a6', content=[ResponseOutputText(annotations=[AnnotationURLCitation(end_index=764, start_index=695, title='NFL Game Scores - - CBS Sports', type='url_citation', url='https://www.cbssports.com/nfl/scoreboard/all/')], text='The last NFL game was the Wild Card playoff between the Los Angeles Rams and the Carolina Panthers, where the Rams won 34-31. This game occurred on January 10, 2026. There were other recent playoff games as well, including the Bears beating the Packers 31-27 and the Bills

In [23]:
run_search(
    "What happened in the last NFL game?",
    tools=[{"type": "web_search_preview"}],
)

In the Verona vs Bologna match on January 15, 2026, Bologna won with a score of 3-1. 

- Gift Orban scored first for Verona in the 13th minute.
- Riccardo Orsolini equalized for Bologna at the 21st minute.
- Jens Odgaard then put Bologna ahead with a goal in the 29th minute.
- Santiago Castro sealed the victory for Bologna in the 44th minute.

The match took place at Stadio Marc'Antonio Bentegodi in Verona. Verona had 38% ball possession and Bologna 62%. The game was part of the Serie A league.

Referee: Maurizio Mariani

This result means Bologna secured a win away from home against Verona with a 3-1 scoreline.[Sky Sports](https://www.skysports.com/football/verona-vs-bologna/live/530208), [ESPN](https://www.espn.com/football/match/_/gameId/736941), [Soccerway](https://us.soccerway.com/game/bologna-0M9xNN8N/verona-rJVAIaHo/)

Citations:

[1] [Sky Sports](https://www.skysports.com/football/verona-vs-bologna/live/530208)
    URL: https://www.skysports.com/football/verona-vs-bologna/live/

Response(id='resp_0693eb11a506a1970069694d178ac08195ad65d442e9a02ec8', created_at=1768508695.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4.1-mini', object='response', output=[ResponseFunctionWebSearch(id='ws_0693eb11a506a1970069694d17c3ec8195a570415de95e7591', action=ActionSearch(query='Verona vs Bologna latest match result', type='search', sources=None), status='searching', type='web_search_call'), ResponseOutputMessage(id='msg_0693eb11a506a1970069694d1895cc8195893558930aebeb23', content=[ResponseOutputText(annotations=[AnnotationURLCitation(end_index=697, start_index=619, title='Verona vs Bologna - Live match updates | 15.01.2026 - Sky Sports', type='url_citation', url='https://www.skysports.com/football/verona-vs-bologna/live/530208'), AnnotationURLCitation(end_index=758, start_index=699, title='Verona vs. Bologna (Jan 15, 2026) Live Score - ESPN', type='url_citation', url='https://www.espn.com/football/match/_/gameId/736941'), AnnotationURLCit

In [19]:
run_search(
    "What are the top 5 renewable energy trends in 2026, and which companies are leading in solar and wind adoption globally?",
    tools=[{"type": "web_search_preview"}],
)


The top 5 renewable energy trends in 2026 are:

1. **Energy Storage Breakthroughs:** Wider use of advanced batteries like lithium-silicon and sodium-ion, and more grid-scale and long-duration energy storage (LDES) to ensure 24/7 clean energy supply.

2. **Solar Power Innovations:** Commercialization of perovskite solar cells for higher efficiency at lower cost, growth in floating solar farms especially in Asia and Europe, and more integration of solar materials into buildings (Building-Integrated Photovoltaics, BIPV).

3. **Green Hydrogen Expansion:** More national hydrogen strategies, reduced electrolyzer costs, and hydrogen hubs in Europe, Asia, and the Middle East enabling decarbonization of hard-to-abate sectors.

4. **Offshore Wind Growth:** Major projects in the North Sea, U.S. East Coast, and Asia-Pacific; growth in floating offshore wind technology for deeper waters; and hybrid systems combining wind with hydrogen or solar.

5. **Smart Grids and AI Integration:** Digital transf

Response(id='resp_0d0f8ef081c7d4640069694087c2fc8195ba36f0547fc2d5cd', created_at=1768505479.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4.1-mini', object='response', output=[ResponseFunctionWebSearch(id='ws_0d0f8ef081c7d46400696940880414819587ccad29ee15b744', action=ActionSearch(query='top renewable energy trends 2026', type='search', sources=None), status='searching', type='web_search_call'), ResponseOutputMessage(id='msg_0d0f8ef081c7d4640069694089dcb8819583c83fa7994c1b77', content=[ResponseOutputText(annotations=[AnnotationURLCitation(end_index=2138, start_index=2022, title='What Are the Top Trends in Renewable Energy for 2026?', type='url_citation', url='https://www.renewableenergyconference.org/blog/trends-in-renewable-energy-2026.html'), AnnotationURLCitation(end_index=2307, start_index=2192, title='Five trends that will shape renewable energy in 2026', type='url_citation', url='https://pv-magazine-usa.com/2025/10/30/five-trends-that-will-sh

In [ ]:
run_search(
    "What are the top 5 renewable energy trends in 2026, and which companies are leading in solar and wind adoption globally?",
    tools=[{"type": "web_search_preview"}],
)


## Top 5 renewable energy trends shaping **2026**
1. **Hybrid renewables (solar/wind + batteries) becoming the default**  
   Utility-scale projects are increasingly paired with 2–4+ hour storage to firm output and capture higher-value evening/peak pricing, with continued growth in grid-scale storage buildouts. 

2. **Grid connection, transmission, and “flexibility” as the binding constraint**  
   The main limiter in many markets is no longer generation cost—it’s interconnection queues, transmission buildout, and system flexibility (storage, demand response, fast-ramping assets). 

3. **Solar keeps leading globally, but growth shifts by region and policy design**  
   Solar remains the largest driver of new renewable additions; however, market growth is uneven (high concentration in China), and policy/market-design changes can create short-term volatility (including a likely global growth dip around 2026 per SolarPower Europe). 

4. **Offshore wind: reset + selective acceleration (esp

Response(id='resp_035094baeb3c3d0c0069686c508554819094b9c845b1df9861', created_at=1768451152.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-5.2', object='response', output=[ResponseFunctionWebSearch(id='ws_035094baeb3c3d0c0069686c513c248190b9a877520128d884', action=ActionSearch(query='2026 renewable energy trends solar wind grid batteries hydrogen', type='search', sources=None), status='completed', type='web_search_call'), ResponseFunctionWebSearch(id='ws_035094baeb3c3d0c0069686c58cee48190bcaeecde1c84d29f', action=ActionOpenPage(type='open_page', url=None), status='completed', type='web_search_call'), ResponseFunctionWebSearch(id='ws_035094baeb3c3d0c0069686c5c1e1481909e3654c3e50faa45', action=ActionSearch(query='IEA Renewables 2025 key findings grid integration financing manufacturing trends', type='search', sources=None), status='completed', type='web_search_call'), ResponseFunctionWebSearch(id='ws_035094baeb3c3d0c0069686c61780481908ad5e1d7190d59eb'

### Location-aware lookup
Optionally bias results by providing an approximate country code.


In [9]:
location_tools = [
    {
        "type": "web_search_preview",
        "user_location": {"type": "approximate", "country": country or "US"},
    }
]

run_search(
    "Share a positive news story from the web today.",
    tools=location_tools,
)


Here is a positive news story from today:

An inspiring story from the Good News Network highlights how an 11-year-old's White House dream came true, and 30 years later, he paid it forward in a heartwarming full-circle moment. This charming story celebrates the power of dreams and giving back to the community. You can read more about uplifting stories like this at Good News Network [source](https://www.goodnewsnetwork.org/).

If you want another type of good news or stories from a specific area or topic, just let me know!

Citations:

[1] [source](https://www.goodnewsnetwork.org/)
    URL: https://www.goodnewsnetwork.org/
    Title: Good News, Inspiring, Positive Stories - Good News Network

Tool calls:

Call 1:
{
  "id": "ws_0dcaba71910d977a006966c38755248190a26d0ab18c20fd87",
  "action": {
    "query": "positive news today",
    "type": "search",
    "sources": null
  },
  "status": "searching",
  "type": "web_search_call"
}


Response(id='resp_0dcaba71910d977a006966c386f2488190b4027e8e8cabc610', created_at=1768342406.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4.1-mini', object='response', output=[ResponseFunctionWebSearch(id='ws_0dcaba71910d977a006966c38755248190a26d0ab18c20fd87', action=ActionSearch(query='positive news today', type='search', sources=None), status='searching', type='web_search_call'), ResponseOutputMessage(id='msg_0dcaba71910d977a006966c38823d88190b5aabcc77fbde4e6', content=[ResponseOutputText(annotations=[AnnotationURLCitation(end_index=427, start_index=385, title='Good News, Inspiring, Positive Stories - Good News Network', type='url_citation', url='https://www.goodnewsnetwork.org/')], text="Here is a positive news story from today:\n\nAn inspiring story from the Good News Network highlights how an 11-year-old's White House dream came true, and 30 years later, he paid it forward in a heartwarming full-circle moment. This charming story celebrates t

### Deep research (multi-step)
Includes the `code_interpreter` tool to let the model perform extended reasoning. Requires a deployment that supports `o3-deep-research`.


In [13]:
print(deep_model)

if deep_model:
    run_search(
        "Research What are the top 5 renewable energy trends in 2026, and which companies are leading in solar and wind adoption globally?",
        tools=[
            {"type": "web_search_preview"},
            #{"type": "code_interpreter", "container": {"type": "auto"}},
        ],
        model_override=deep_model,
    )
else:
    print("Skip deep research demo; set AZURE_OPENAI_DEEP_MODEL to enable.")


o3-pro
Top 5 renewable-energy trends to watch in 2026  

1. Long-duration energy storage goes mainstream  
   • Prices for next-generation lithium-silicon, sodium-ion and flow batteries continue to fall, and more than 30 GW of stand-alone or co-located storage is expected to be commissioned worldwide in 2026. Utilities and developers are pairing solar and wind with four-hour-plus batteries to supply “clean-firm” power and to qualify for capacity and grid-services revenues.   

2. Green hydrogen moves from pilots to first-wave commercial hubs  
   • Policy clarity in the EU’s Delegated Acts and multigigawatt auctions in India and the Middle East are triggering final-investment decisions for 100 MW-scale electrolyser projects. Electrolyser costs have dropped below US $500 per kW, opening a viable pathway to decarbonise steel, fertiliser and heavy transport.   

3. Offshore—and now floating—wind scales rapidly  
   • Fixed-bottom additions in China, the North Sea and the U.S. Atlantic wil

In [13]:
run_search(
    "Client:A1292382 " \
    " SSN: 123482102" \
    " holdings: 100 AAPL, 200 NVDA.  Search latest news on my holdings and help me analyze my portfolio risks and suggest diversification strategies.",
    tools=[{"type": "web_search_preview"}]

)


Here is the latest news summary and analysis for your holdings:

**Apple Inc. (AAPL)**  
- Apple recently announced a multi-year partnership with Google to enhance its AI capabilities, specifically improving Siri by integrating Google's Gemini AI models. This collaboration is expected to accelerate Apple's time-to-market for AI features across devices while managing costs efficiently.  
- Strong iPhone sales are anticipated for the upcoming fiscal quarter, potentially boosting Apple's revenue and investor confidence.  
- Analysts are generally bullish on Apple, reaffirming buy ratings with optimistic price targets ahead of the upcoming earnings report on January 29, 2026.  
- Risks include regulatory scrutiny, especially around AI implementations in various regions and ongoing tensions related to tech sector regulations.  
- Overall, Apple maintains a strong market position with a mix of innovation in AI and solid hardware sales.  

**NVIDIA Corporation (NVDA)**  
- NVIDIA showcased ke

Response(id='resp_0894f5757f27638a006966c48d03008190b4d99921b1f53c5c', created_at=1768342669.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4.1-mini', object='response', output=[ResponseFunctionWebSearch(id='ws_0894f5757f27638a006966c48d6a44819098cea8871f58df61', action=ActionSearch(query='latest news AAPL', type='search', sources=None), status='searching', type='web_search_call'), ResponseFunctionWebSearch(id='ws_0894f5757f27638a006966c48e5ea081908c520232a3f1281f', action=ActionSearch(query='latest news NVDA', type='search', sources=None), status='searching', type='web_search_call'), ResponseOutputMessage(id='msg_0894f5757f27638a006966c48f541c8190baa73dcfa2ab6b59', content=[ResponseOutputText(annotations=[], text="Here is the latest news summary and analysis for your holdings:\n\n**Apple Inc. (AAPL)**  \n- Apple recently announced a multi-year partnership with Google to enhance its AI capabilities, specifically improving Siri by integrating Google's

### Domain filtering
Limit web search results to specific domains using the `filters` parameter with an allow-list of up to 100 URLs. When formatting URLs, omit the HTTP/HTTPS prefix (use `openai.com` instead of `https://openai.com/`). This includes subdomains in the search.

The `sources` field returns the complete list of URLs consulted, which is often greater than inline citations. Real-time feeds are labeled as `oai-sports`, `oai-weather`, or `oai-finance`.

In [12]:
filtered_tools = [
    {
        "type": "web_search_preview",
        "filters": {
            "allowed_domains": [
                "pubmed.ncbi.nlm.nih.gov",
                "clinicaltrials.gov",
                "www.who.int",
                "www.cdc.gov",
                "www.fda.gov"
            ]
        }
    }
]

response = run_search(
    "Please perform a web search on how semaglutide is used in the treatment of diabetes.",
    tools=filtered_tools,
)

# Display sources if available
if hasattr(response, 'output') and response.output:
    sources = []
    for item in response.output:
        if hasattr(item, 'sources') and item.sources:
            sources.extend(item.sources)
    
    if sources:
        print("\n" + "=" * 80)
        print(f"Sources consulted ({len(sources)} total):")
        print("=" * 80)
        for i, source in enumerate(sources, 1):
            print(f"{i}. {source}")
        print("=" * 80)

BadRequestError: Error code: 400 - {'error': {'message': "Unsupported parameter 'filters'", 'type': 'invalid_request_error', 'param': 'tools', 'code': None}}